## DataSet

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from torchvision import transforms
from glob import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import itertools
import numpy as np
from PIL import Image
T = torch.Tensor
rand_m = np.random.random

In [2]:
from data import Tiny_imagenet_dataset

In [3]:
tt = Tiny_imagenet_dataset()

In [4]:
train_loader = DataLoader(tt, batch_size=32, shuffle=True)

In [5]:
for x, y in train_loader:
    break

In [6]:
x[0], y

(tensor([[[-0.6794,  0.5364, -1.2617,  ..., -1.0733, -1.3302, -1.4329],
          [-1.0219, -0.9363, -1.0562,  ..., -0.9020, -1.1932, -1.2959],
          [-0.6623, -0.6623, -0.0972,  ..., -1.1932, -1.3130, -1.2788],
          ...,
          [ 0.5022,  1.1015,  0.4508,  ..., -0.4054, -0.4739, -0.3027],
          [ 2.0263,  0.2111,  1.6324,  ...,  0.2624,  0.3138,  0.0912],
          [ 1.8550,  1.2385,  0.5193,  ...,  0.4166,  0.0912, -0.0287]],
 
         [[-0.1625,  1.0805, -0.7577,  ..., -0.7752, -1.0378, -1.1429],
          [-0.5476, -0.4601, -0.6001,  ..., -0.5651, -0.8627, -0.9678],
          [-0.2850, -0.2850,  0.3102,  ..., -0.8627, -0.9853, -0.9503],
          ...,
          [ 0.1352,  0.7479,  0.1176,  ..., -0.6527, -0.7227, -0.5476],
          [ 1.6583, -0.1625,  1.3256,  ...,  0.0301,  0.0826, -0.1450],
          [ 1.4832,  0.8880,  0.1877,  ...,  0.1877, -0.1450, -0.2675]],
 
         [[-0.0441,  1.1585, -0.6715,  ..., -0.8458, -1.1073, -1.2119],
          [-0.4450, -0.3578,

## define model and training loop


In [7]:
from torchvision import models

In [8]:
class r_net(nn.Module):
    def __init__(self):
        super(r_net, self).__init__()
        
        self.resnet = models.resnet34(pretrained=True)
        
        self.avgp = nn.AdaptiveAvgPool2d(1)
        self.final_conv = nn.Conv2d(512, 200, 1)
        
    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)
        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)
        x = self.avgp(x)
        x = self.final_conv(x)
        x = x.view(x.shape[0], -1)
        return x

r = r_net().cuda()
r.training

True

In [9]:
r(torch.Tensor(np.random.random((32, 3, 64, 64))).cuda()).shape

torch.Size([32, 200])

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(r.parameters())

In [ ]:
for epoch in range(20):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        if i % 200 == 0: 
            print("-- ITERATION:", i)
        input, target = data

        input_var = input.cuda()
        target_var = target.cuda()
        
        optimizer.zero_grad()
        
        output = r(input_var)
        
        pred = output
        pred = torch.max(pred, 1)[1]
        
        
        loss = criterion(output, target_var)
        
        # computer gradient + sgd step
        loss.backward()
        
        #for param in r.parameters():
        #    print(param.grad.data.sum())
        
        optimizer.step()

        # print progress
        running_loss += loss.item()
            
            
        if i % 200 == 0:
            print(running_loss / 50)
            running_loss = 0.
            pred = output
            pred = torch.max(pred, 1)[1]
            acc = (pred == target_var).float().mean()
            print(acc.item())

-- ITERATION: 0
0.11802024841308593
0.0
